Doublet detection with "DoubletDetection"

INFO: Run with conda environment sc-mar2021

This is the script/notebook I used for the actual processing.

# 0. Load packages

In [1]:
import os
import sys
import glob
import re

import anndata
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import doubletdetection

import logging

In [2]:
# General settings
plt.rcParams['figure.figsize']=(20,12) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()

-----
anndata     0.7.5
scanpy      1.8.2
sinfo       0.3.1
-----
PIL                 8.1.2
anndata             0.7.5
anyio               NA
attr                20.3.0
babel               2.9.0
backcall            0.2.0
brotli              NA
cairo               1.19.1
certifi             2022.12.07
cffi                1.14.5
chardet             4.0.0
colorama            0.4.5
constants           NA
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.1
decorator           4.4.2
doubletdetection    3.0
google              NA
h5py                3.7.0
highs_wrapper       NA
idna                2.10
igraph              0.9.1
ipykernel           5.3.4
ipython_genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
jinja2              2.11.3
joblib              1.0.1
json5               NA
jsonschema          3.2.0
jupyter_server      1.4.1
jupyterlab_server   2.3.0
kiwisolver          1.3.1
leidenalg           0.8.4
llvmlite            0.35.0
louvain   

In [3]:
sys.path.insert(0,'..')
import paths_downsampling as paths
p = paths.get_paths()
print(p)

{'basedir': '/psycl/g/mpsagbinder/mgp/workspace/SingleNuc_PostmortemBrain/', 'rawdir_RNA': '/psycl/g/mpsngs/HiSeq_Helmholtz/20210324_Anna_Froehlich_10X_RNAseq/03_downsampled/', 'figdir': '/psycl/g/mpsagbinder/mgp/workspace/SingleNuc_PostmortemBrain/figures/', 'writedir': '/psycl/g/mpsagbinder/mgp/workspace/SingleNuc_PostmortemBrain/scanpy_adata/', 'allendir': '/psycl/g/mpsagbinder/mgp/workspace/SingleNuc_PostmortemBrain/reference_data/allen_human/'}


# 1. Load data

In [4]:
adata = sc.read(p['writedir']+'adata_raw_qc_RNA_downsampling_perCell.h5ad')

# 2. Run doublet detection on each sample

In [ ]:
adatas = {}
for s in adata.obs['sample'].unique():
    a = adata[adata.obs['sample'] == s, :]
    
    #clf = doubletdetection.BoostClassifier(
    #    n_iters=25, 
    #    use_phenograph=False, 
    #    standard_scaling=True
    #)
    #a.obs['dd_doublet'] = clf.fit(a.X).predict(p_thresh=1e-16, voter_thresh=0.5)
    #a.obs['dd_score'] = clf.doublet_score()
    
    clf = doubletdetection.BoostClassifier()
    # raw_counts is a cells by genes count matrix
    a.obs['dd_doublet'] = clf.fit(a.X).predict()
    # higher means more likely to be doublet
    a.obs['dd_score'] = clf.doublet_score()
    
    adatas[s] = a

In [6]:
# restructure from map to list
adatas = [adatas[k] for k in adatas.keys()]
#print(adatas)

In [7]:
# concatenate cells from different samples into one matrix
adata = adatas[0].concatenate(adatas[1:])
#del adatas
print(adata.var)

/u/ngerst/anaconda3/envs/sc-mar2021/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


                   gene_ids   mito  mean_counts  log1p_mean_counts  \
AL627309.1  ENSG00000238009  False     0.004863           0.004851   
AL627309.5  ENSG00000241860  False     0.012233           0.012159   
LINC01409   ENSG00000237491  False     0.093638           0.089510   
LINC01128   ENSG00000228794  False     0.152879           0.142262   
LINC00115   ENSG00000225880  False     0.007871           0.007840   
...                     ...    ...          ...                ...   
AL592183.1  ENSG00000273748  False     0.241273           0.216137   
AC240274.1  ENSG00000271254  False     0.020337           0.020133   
AC004556.3  ENSG00000276345  False     0.001587           0.001586   
AC007325.4  ENSG00000278817  False     0.019940           0.019744   
AC007325.2  ENSG00000277196  False     0.006018           0.006000   

            pct_dropout_by_counts  total_counts  log1p_total_counts  n_cells  \
AL627309.1              99.518325        4356.0            8.379539     4316   

# 3. Visualize results

## 3.1 Doublets on UMAP

In [8]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata)
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

normalizing counts per cell
    finished (0:00:10)
extracting highly variable genes
    finished (0:00:51)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:02:22)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:02:24)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:46:16)


In [ ]:
sc.pp.calculate_qc_metrics(adata, inplace=True)
adata.obs

In [10]:
adata.obs['dd_doublet'] = adata.obs['dd_doublet'].astype('category')
sc.pl.umap(adata, color=["dd_doublet", "dd_score"])
sc.pl.umap(adata, color=['total_counts', 'n_genes_by_counts'])

... storing 'sample' as categorical
... storing 'AgeBin' as categorical
... storing 'Sex' as categorical
... storing 'Classification' as categorical
... storing 'Classification.detail' as categorical
... storing 'Trauma.notes.medical.history' as categorical
... storing 'Hemisphere' as categorical
... storing 'Chlorpromazine.equivalent' as categorical
... storing 'Antidepressants' as categorical
... storing 'Mode.of.death' as categorical
... storing 'Manner.of.Death' as categorical
... storing 'COD.category' as categorical
... storing 'positive.Toxicology' as categorical
... storing 'Antipsychotics' as categorical
... storing 'Antipsychotics.meds.pres' as categorical
... storing 'Smoking.status' as categorical
... storing 'Group' as categorical
... storing 'aaa' as categorical


## 3.2 Doublet statistics

In [11]:
sc.pl.violin(adata, "dd_score")

# 4. Write results

In [12]:
# Save data to file
adata.write(p['writedir']+'adata_qc_doubletdetection_Downsampling_RNA.h5ad')